Test du projet de class trajets indirect

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import traitement_od as t
import pandas as pd
import numpy as np
import altair as alt
import datetime as dt
pd.set_option('display.max_colwidth', 1000)
import os
from sklearn.cluster import DBSCAN

IMPORT DONNEES GLOBALES (SUPPRESSION DES DOUBLONS EN MM TEMPS)

In [353]:
%%time df_3semaines=t.ouvrir_fichier_lapi('2019-01-28 00:00:00','2019-02-17 23:59:59')
df_3semaines=df_3semaines.drop_duplicates(['created','immat']))
df_3semaines.set_index('created').sort_index()

Wall time: 2min 3s


IMPORT DES DONNEES SUR UNE JOURNEE

In [350]:
df_journee=t.ouvrir_fichier_lapi('2019-01-28','2019-01-30')

In [352]:
#redresser la valeur de l
df_journee.loc[df_journee['camera_id']==15,'l']=1

LISTE DES TRAJETS

In [3]:
liste_trajet_locale=(pd.DataFrame([{'origine':'A63','destination':'A10','cam_o':15, 'cam_d':11, 'trajets':[
                                                        {'cameras':[15,19,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A63','cam_o':12, 'cam_d':13,'trajets':[{'cameras':[12,6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'N10','cam_o':15, 'cam_d':5,'trajets':[{'cameras':[15,19,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,5],'type_trajet':'indirect'}, 
                                                        {'cameras':[15,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A63','cam_o':6, 'cam_d':13,'trajets':[{'cameras':[6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A10','cam_o':10, 'cam_d':11,'trajets':[{'cameras':[10,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A62','cam_o':12, 'cam_d':9,'trajets':[{'cameras':[12,6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'N10','cam_o':10, 'cam_d':5,'trajets':[{'cameras':[10,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[10,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A62','cam_o':6, 'cam_d':9,'trajets':[{'cameras':[6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[6,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A62','cam_o':15, 'cam_d':9,'trajets':[{'cameras':[15,19,9],'type_trajet':'indirect'},
                                                        {'cameras':[15,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A63','cam_o':10, 'cam_d':13,'trajets':[{'cameras':[10,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[10,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A63','cam_o':8 ,'cam_d':13,'trajets':[{'cameras':[8,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A89','cam_o':15, 'cam_d':7,'trajets':[{'cameras':[15,19,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,7],'type_trajet':'indirect'},
                                                        {'cameras':[15,7],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A62','cam_o':8, 'cam_d':9,'trajets':[{'cameras':[8,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[8,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'A89','cam_o':10, 'cam_d':7,'trajets':[{'cameras':[10,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[10,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'A10','cam_o':8, 'cam_d':11,'trajets':[{'cameras':[8,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[8,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A89','cam_o':12, 'cam_d':7,'trajets':[{'cameras':[12,6,7],'type_trajet':'indirect'},
                                                        {'cameras':[12,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'N10','cam_o':8, 'cam_d':5,'trajets':[{'cameras':[8,5],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'N10','destination':'A89','cam_o':6, 'cam_d':7,'trajets':[{'cameras':[6,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A660','cam_o':12, 'cam_d':18,'trajets':[{'cameras':[12,6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A660','destination':'A10','cam_o':19, 'cam_d':18,'trajets':[{'cameras':[19,4,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,11],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A660','destination':'A62','cam_o':19, 'cam_d':9,'trajets':[{'cameras':[19,9],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A62','destination':'A660','cam_o':10, 'cam_d':18,'trajets':[{'cameras':[10,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A660','destination':'A89','cam_o':19, 'cam_d':7,'trajets':[{'cameras':[19,4,7],'type_trajet':'indirect'},
                                                         {'cameras':[19,7],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A89','destination':'A660','cam_o':8, 'cam_d':18,'trajets':[{'cameras':[8,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[8,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'N10','destination':'A660','cam_o':6, 'cam_d':18,'trajets':[{'cameras':[6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A660','destination':'N10','cam_o':19, 'cam_d':5,'trajets':[{'cameras':[19,1,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,5],'type_trajet':'direct'},
                                                        ]}
                           ]))[['origine', 'destination', 'cam_o', 'cam_d','trajets']]

In [730]:
%time test=t.trajet(df_journee, '2019-01-29 08:00:00', 60, 16, [6,13], type='Global')

Wall time: 6.33 s


CALCUL DES O-D SUR UNE JOURNEE, AVEC RENVOI DES DF O-D OU PASSAGES DANS UN DICO PAR O-D OU UNE DF GLOBALE

In [353]:
dico_trajet_od, dico_passag_od, dico_od,  dico_passag=t.transit_1_jour(df_journee,'2019-01-28 08:00:00', liste_trajet_locale)

date : 2019-01-28 08:00:00 debut_traitement : 2019-04-05 15:32:42.058239
date : 2019-01-28 09:00:00 debut_traitement : 2019-04-05 15:35:25.209336


IMPORTANT : VERIF DES DOUBLONS ENTRE LES TRAJETS

In [10]:
A660N10=dico_passag_od['A660-N10'].set_index(['created','immat'])
A63N10=dico_passag_od['A63-N10'].set_index(['created','immat'])
A63N10.loc[A63N10.index.isin(A660N10.index)].reset_index().set_index('created')

,immat,camera_id,fiability,l,state
created,,,,,


CREER LE TABLEAU DE MATRICE O-D

In [358]:
#MATRICE O-D !!II!
pd.pivot_table(dico_od,values='l', index='origine', columns='destination',aggfunc='count', margins=True )

destination,A10,A62,A63,A660,A89,N10,All
origine,,,,,,,
A10,NaN,6255.0,1849.0,3185.0,297.0,NaN,11586
A62,1221.0,NaN,285.0,1010.0,510.0,2691.0,5717
A63,1441.0,158.0,NaN,NaN,806.0,16914.0,19319
A660,NaN,1020.0,NaN,NaN,1948.0,11462.0,14430
A89,373.0,1225.0,1313.0,2498.0,NaN,1453.0,6862
N10,NaN,4339.0,16666.0,10552.0,855.0,NaN,32412
All,3035.0,12997.0,20113.0,17245.0,4416.0,32520.0,90326


POURCENTAGE PL PAR PLAGE HORAIRE ET CAMERA

In [432]:
#isoler pl source
groupe_tv_type_simple=df_journee.groupby('immat')['l'].apply(lambda x:test_unicite_type(list(x),mode='1/2')).reset_index().drop_duplicates('immat')
df_pl=df_journee.join(groupe_tv_type_simple.loc[groupe_tv_type_simple['l']==1].set_index('immat'),on='immat',how='inner',rsuffix='_right')
df_pl.set_index('created',inplace=True)

#obtenir les nb de pl par heure et par camera sur la source
df_synthese_pl_tot=df_pl.groupby('camera_id').resample('H').count()['immat'].rename(column={'immat':'nb_pl_tot'})
df_synthese_pl_transit=dico_passag.set_index('created').groupby('camera_id').resample('H').count()['immat'].rename(column={'immat':'nb_pl_transit'})
df_pct_pl_transit=pd.concat([df_synthese_pl_tot,df_synthese_pl_transit], axis=1, join='inner')
df_pct_pl_transit.columns=[['nb_pl_tot','nb_pl_transit']]
df_pct_pl_transit['pct_pl_transit']=df_pct_pl_transit.apply(lambda x : float(x['nb_pl_transit'])*100 / x['nb_pl_tot'] ,axis=1)
df_pct_pl_transit

TESTS DONNEES COMPLETES

In [357]:
dico_trajet_od, dico_passag_od, dico_od,  dico_passag, dico_filtre=t.transit_temps_complet_v2('2019-01-28 00:00:00',21,liste_trajet_locale,df_3semaines)

date : 2019-01-28 00:00:00 debut_traitement : 2019-04-08 17:23:56.903318
date : 2019-01-28 01:00:00 debut_traitement : 2019-04-08 17:24:23.256536
date : 2019-01-28 02:00:00 debut_traitement : 2019-04-08 17:24:53.185116
date : 2019-01-28 03:00:00 debut_traitement : 2019-04-08 17:25:20.202851
date : 2019-01-28 04:00:00 debut_traitement : 2019-04-08 17:25:46.649768
date : 2019-01-28 05:00:00 debut_traitement : 2019-04-08 17:26:11.690765
date : 2019-01-28 06:00:00 debut_traitement : 2019-04-08 17:27:01.788352
date : 2019-01-28 07:00:00 debut_traitement : 2019-04-08 17:28:20.238716
date : 2019-01-28 08:00:00 debut_traitement : 2019-04-08 17:29:54.857130
date : 2019-01-28 09:00:00 debut_traitement : 2019-04-08 17:31:39.520041
date : 2019-01-28 10:00:00 debut_traitement : 2019-04-08 17:33:16.341061
date : 2019-01-28 11:00:00 debut_traitement : 2019-04-08 17:34:40.557102
date : 2019-01-28 12:00:00 debut_traitement : 2019-04-08 17:35:56.695510
date : 2019-01-28 13:00:00 debut_traitement : 2019-

In [388]:
#verifs
dico_passag_od['A660-N10'].set_index('created').loc['2019-01-31 08:00:00':].sort_index().head(10) #visualiser les passages

,camera_id,immat,fiability,l,state
created,,,,,
2019-01-31 08:00:30,19,9dd7529a19c83bed931c0d4b40840b7a36328e20,99,1,ES
2019-01-31 08:01:15,19,b7e7d7976213d905a87691099965260a3474b2bd,99,-1,ES
2019-01-31 08:01:30,19,15b4f08f0d66aa2205d4c74c66e7a1fcff1d5852,99,-1,FR
2019-01-31 08:01:56,19,574c4311fa597da55a2aac80e1097129dd7419e1,99,-1,FR
2019-01-31 08:02:15,19,c57c4941aadcb97905b3ef0e410578f22f516efa,99,-1,ES
2019-01-31 08:02:38,19,8ddc4c2f1189ae0a4224f688200b211c3fd0f675,99,1,ES
2019-01-31 08:02:49,19,d735cccf1331cd23ba652c145358f46405bbc306,99,0,ES
2019-01-31 08:03:30,19,d4ea00db5e41cf4e295e2d36530168ecd2e0b696,89,0,RO
2019-01-31 08:04:14,19,e5bf70ce74cc64bf07a4fbcbe56e0e35d267b970,99,0,ES


In [390]:
df_3semaines.loc[df_3semaines['immat']=='e5bf70ce74cc64bf07a4fbcbe56e0e35d267b970']#retrouver cette immatriculatiob dans les données completes

,camera_id,immat,fiability,l,state
created,,,,,
2019-01-31 08:04:14,19,e5bf70ce74cc64bf07a4fbcbe56e0e35d267b970,99,0,ES
2019-01-31 08:47:59,5,e5bf70ce74cc64bf07a4fbcbe56e0e35d267b970,99,1,ES
2019-02-02 11:40:52,6,e5bf70ce74cc64bf07a4fbcbe56e0e35d267b970,99,1,ES
2019-02-02 11:52:45,3,e5bf70ce74cc64bf07a4fbcbe56e0e35d267b970,99,0,ES
2019-02-02 12:12:01,18,e5bf70ce74cc64bf07a4fbcbe56e0e35d267b970,99,1,ES
2019-02-11 08:33:27,4,e5bf70ce74cc64bf07a4fbcbe56e0e35d267b970,99,1,ES
2019-02-11 08:44:31,5,e5bf70ce74cc64bf07a4fbcbe56e0e35d267b970,99,1,ES


In [354]:
#verif sur un trajet a un horaire donnee sur un trajet global
%time test=t.trajet(df_3semaines.loc['2019-01-29 08:00:00':'2019-01-30 15:00:00'], '2019-01-29 08:00:00', 60, 16, [15,5], type='Global')

Wall time: 17.1 s


In [ ]:
#verif a un horaire donne sur un trajet simple
%time test=t.trajet(df_3semaines.loc['2019-01-29 21:00:00':'2019-01-30 15:00:00'], '2019-01-29 21:00:00', 60, 16, [19,4])

In [135]:
#pour test des fonctions de trajets directs
df_duree=df_3semaines.loc['2019-01-29 21:00:00':'2019-01-30 15:00:00'].loc['2019-01-29 21:00:00':'2019-01-29 22:00:00']
df_duree_cam1=df_duree.loc[df_duree.loc[:,'camera_id']==19]
df_duree_autres_cam=df_3semaines.loc['2019-01-29 21:00:00':'2019-01-30 15:00:00'].loc[(df_3semaines.loc['2019-01-29 21:00:00':'2019-01-30 15:00:00'].loc[:,'immat'].isin(df_duree_cam1.loc[:,'immat']))]
cam1_croise_autre_cam=df_duree_cam1.reset_index().merge(df_duree_autres_cam.reset_index(), on='immat')
cam1_croise_suivant=cam1_croise_autre_cam.loc[(cam1_croise_autre_cam.loc[:,'created_x']<cam1_croise_autre_cam.loc[:,'created_y'])]
cam1_fastest_next=cam1_croise_suivant.loc[cam1_croise_suivant.groupby(['immat'])['created_y'].idxmin()]

In [319]:
#pour test des fonctions de trajets global 
liste_trajet_od=[(15,19,4,5),(15,5),(15,4,5),(15,19,5)]
camera1, camera2=15, 5
df=df_3semaines.loc['2019-01-29 00:00:00':'2019-01-30 23:59:59']
df_duree=df.loc['2019-01-29 08:00:00':'2019-01-29 09:00:00']
df_duree_cam1=df_duree.loc[df_duree.loc[:,'camera_id']==camera1]
df_duree_autres_cam=df.loc[(df.loc[:,'immat'].isin(df_duree_cam1.loc[:,'immat']))]
df_autres_cam_temp_ok=df_duree_autres_cam.loc['2019-01-29 08:00:00':'2019-01-30 08:00:00']
groupe=(df_autres_cam_temp_ok.sort_index().reset_index().groupby('immat').agg({'camera_id':lambda x : tuple(x), 'l': lambda x : test_unicite_type(list(x),'1/2'),
                                                                        'created':lambda x: tuple(x)}))

groupe['camera_id']=groupe.apply(lambda x : filtrer_passage(x['camera_id'],liste_trajet_od),axis=1)

groupe['created']=groupe.apply(lambda x : recuperer_date_cam2(x['camera_id'],x['created'],liste_trajet_od),axis=1)

df_agrege=df_duree_cam1.join(groupe,on='immat',lsuffix='_left')[['immat', 'l','camera_id','created']].rename(columns={'created':'date_cam_2', 'camera_id':'cameras'})
df_agrege=df_agrege.reset_index().rename(columns={'created':'date_cam_1'})
df_agrege['tps_parcours']=df_agrege.apply(lambda x : x.date_cam_2-x.date_cam_1, axis=1)

df_trajet=df_agrege.loc[df_agrege['cameras'].isin(liste_trajet_od)]
df_trajet_final=(df_trajet.loc[(df_trajet['cameras'].apply(lambda x : x[-1])==camera2) & (df_trajet['l']==1)
                                  & (df_trajet['tps_parcours'] < pd.Timedelta('11:30:00'))])

In [269]:
def test_unicite_type(liste_l, mode='unique'):
        """test pour voir si un vehicule a ete toujours vu de la mme façon ou non
           en entre : liste de valeur de l iisues d'une df""" 
        if mode=='unique' : 
            if len(set(liste_l))==1 :
                return liste_l[0]
            else : 
                return -1
        elif mode=='1/2' :
            if any(liste_l)==1 : 
                return 1
            else : 
                return -1

In [349]:
test.temps_parcours_max

Timedelta('0 days 13:02:50')

16058